In [331]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_percentage_error

## Objectif du projet : 
#### Prédire le nombre de validations par jour pour toutes les gares de SNCF-Transilien entre le 1er janvier 2023 et le 30 juin 2023. 


#### x-train est un jeu de données avec 1 237 971 lignes et 6 colonnes. Il s’agit des données quotidiennes de validations des 448 stations du réseau SNCF-Transilien (comprenant toutes les gares du RER A et du RER B) entre le 1er janvier 2015 et le 31 décembre 2022 (soit 2922 jours).

#### et y-train c'est le nombre de validations par jour pour toutes les gares de SNCF-Transilien entre le 1er janvier 2015 et le 31 décembre 2022 (soit 2922 jours).

In [332]:
#Récupération des données
X = pd.read_csv("Data/train_f_x.csv")
y =  pd.read_csv("Data/y_train_sncf.csv")
final_test_data = pd.read_csv("Data/test_f_x_THurtzP.csv")

In [333]:
# Convertir les dates en nombre de jour depuis le début de l'enregistrement

def days_since_start(df) : 
    df['date'] = pd.to_datetime(df['date'])
    # Convertir les dates en jours écoulés depuis une date de référence
    df['days_since_start'] = (df['date'] - pd.to_datetime("2015-01-01")).dt.days  
    return df  



In [334]:
# Convertir les dates en jours écoulés depuis une date de référence
X = days_since_start(X)
final_test_data  = days_since_start(final_test_data)


In [335]:
#Selection des variables indépendantes :
X = X[["days_since_start","station","job", "ferie","vacances"]]
final_test_data = final_test_data[["days_since_start","station","job", "ferie","vacances"]]

# J'enlève les jours dont le nombre de validation dans la gare est nulle car c'est presque impossible que ç'arrive
valeurs_manquantes = y.loc[y["y"]==0]
indices_to_exclure = valeurs_manquantes.index
X = X.drop(indices_to_exclure)
y = y.drop(indices_to_exclure)
#Selection des variables à préduire, j'enlève les indices 
y = y["y"]


In [336]:
#Création de l'ensemble de test et l'ensemble d'entrainement:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [337]:
#Création d'une pipeline pour la transformation de la colonne station et la réalisation du modèle

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), ['station'])
    ],
    remainder='passthrough'
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])


# Entraîner le modèle
pipeline.fit(X_train, y_train)

# Prédire sur l'ensemble de test
y_pred = pipeline.predict(X_test)

# Évaluer le modèle
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')



Mean Squared Error: 21478450.557135094
R^2 Score: 0.7743547572951


In [338]:
mape = mean_absolute_percentage_error(y_test,y_pred)
mape

12.940642867999314

In [355]:

pred = pipeline.predict(final_test_data.head(20))

array([-4724.00118601, -4861.86485645, -4771.19934986, -4762.12218729,
       -4680.58604855, -4481.66780853, -4637.16992733, -2142.44990791,
       -4526.94270232, -4303.65650072, -4648.07836569, -4476.7832296 ,
       -4459.89185418, -3785.99285395, -4244.16501406, -4728.30678918,
       -4473.75771482, -4417.20217792, -4044.1007875 , -4825.34288979])

data